# Instalar dependencias

In [ ]:
pip install overpy

In [35]:
pip install fsspec

# Cargar módulos e inicializar

In [ ]:
import requests
import overpy

api = overpy.Overpass()

# Comprobar estado de las connexiones a Overpass

*La API de Overpass, tiene una limitación en cuanto a la cantidad de consultas recurrentes que se pueden realizar y en algunos casos puede generar un timeout con el error **OverpassTooManyRequests**.*

*Podemos comprobar el estado de nuestra connexión con el siguiente fragmento:*

In [ ]:
estado = requests.get('https://overpass-api.de/api/status')
print(estado.content.decode('utf-8'))

Connected as: 576445613
Current time: 2021-05-04T20:11:32Z
Rate limit: 2
2 slots available now.
Currently running queries (pid, space limit, time limit, start time):



# Consulta

En la primera línea de la consulta podemos indicar la fecha.

La seguna línea sirve para limitar los resultados a España.

A continuación se recogen distintas categorías de comercio, según la clave amenities. He añadido las que me han parecido que encajaban con lo solicitado. Puedes ver todos los valores posibles de la clave amenities en https://wiki.openstreetmap.org/wiki/Key:amenity.

In [ ]:
query = ('[out:json][timeout:25000][date:"2020-12-31T23:59:59Z"];'
           'area["name"="España"][admin_level="2"];('
              'node["amenity"="bar"](area); way["amenity"="bar"](area);'
              'node["amenity"="biergarten"](area); way["amenity"="biergarten"](area);'
              'node["amenity"="cafe"](area); way["amenity"="cafe"](area);'
              'node["amenity"="fast_food"](area); way["amenity"="fast_food"](area);'
              'node["amenity"="food_court"](area); way["amenity"="food_court"](area);'
              'node["amenity"="pub"](area); way["amenity"="pub"](area);'
              'node["amenity"="restaurant"](area); way["amenity"="restaurant"](area);'
           ');out;')
resultado = api.query(query)

*Los resultados devueltos se dividen entre nodes (puntos ) y ways (líneas o polígonos).*

*Algunos comercios y edificios a veces se guardan como polígonos, aunque la mayoría estan definidos como puntos.*

# Examinar el resultado

In [ ]:
print(f'Hay {len(resultado.nodes)} "nodes".')
print(f'Hay {len(resultado.ways)} "ways".')

Hay 41433 "nodes".
Hay 0 "ways".


# Recorrer los puntos devueltos

In [ ]:
for comercio in resultado.nodes:
    # Ver todas las propiedades y métodos de este objeto
    #print(dir(comercio))
    # Ver las propiedades del comercio
    print(comercio.tags)
    # Ver las coordenadas del comercio
    print(comercio.lat, comercio.lon)

NameError: ignored

# Recorrer los polígonos devueltos

In [ ]:
for comercio in resultado.ways:
    # Ver todas las propiedades y métodos de este objeto
    #print(dir(comercio))
    # Ver las propiedades del comercio
    print(comercio.tags)
    # Ver las coordenadas del comercio
    print(comercio.lat, comercio.lon)

# Descargar los resultados

In [36]:
import pandas as pd
from google.colab import files
from io import StringIO
import json
todos = []
for comercio in resultado.nodes:
    data = {
        "name": comercio.tags.get('name', None),
        "lat": float(comercio.lat),
        "lon": float(comercio.lon)
    }
    data.update(comercio.tags)
    todos.append(data)

# Muestra de los primeros 10 registros
df = pd.read_json(StringIO(json.dumps(todos)))
print("Muestra de los 10 primeros registros. {} columnas.".format(len(df.columns)))
print("")
pd.set_option('display.max_columns', 20)
print(df.head(10))
# Descarga
df.to_csv('bares_espana.csv')
files.download('bares_espana.csv')

Muestra de los primeros 10 registros. 267 columnas.

                   name        lat       lon addr:city addr:housenumber  \
0              Mür Café  40.425642 -3.712200    Madrid                2   
1      Cocktails+Dreams  39.655822  3.438309       NaN              NaN   
2  Polynesian Cocktails  39.656648  3.438906       NaN              NaN   
3    Cadillac Solitario  40.487473 -3.699712       NaN              NaN   
4           Bar Barocco  40.413064 -3.712934       NaN              NaN   
5                 Gilda  39.474825 -0.382234       NaN              NaN   
6                   Bar  40.409488 -3.693807       NaN              NaN   
7                  None  40.411638 -3.697940       NaN              NaN   
8                 Fever  40.412445 -3.696158       NaN              NaN   
9             La Hidria  40.412804 -3.695685       NaN              NaN   

  addr:postcode               addr:street amenity             email  \
0         28015  Plaza de Cristino Martos     bar 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>